In [5]:
import functools
import itertools
import requests
import pandas as pd
from tqdm.contrib.concurrent import thread_map

In [6]:
def get_price_range(minPrice, maxPrice, session):
    api_url = f"https://www.immoweb.be/en/search-results/house-and-apartment/for-sale?countries=BE&page=1&orderBy=newest&isALifeAnnuitySale=false&minPrice={minPrice}&maxPrice={maxPrice}"
    num_pages = session.get(api_url).json()['marketingCount'] // 30 + 1
    if num_pages > 330:
        mid_price = (minPrice + maxPrice) // 2 
        return get_price_range(minPrice, mid_price, session).union(get_price_range(mid_price, maxPrice, session))
    return get_ids_for_category(minPrice, maxPrice, num_pages, session)

def get_ids_from_page(i, minPrice, maxPrice, session):
    api_url = f"https://www.immoweb.be/en/search-results/house-and-apartment/for-sale?countries=BE&page={i}&orderBy=newest&isALifeAnnuitySale=false&minPrice={minPrice}&maxPrice={maxPrice}"
    return set(result['id'] for result in session.get(api_url).json()['results'])

def get_ids_for_category(minPrice, maxPrice, num_pages, session):
    return set(itertools.chain.from_iterable(thread_map(functools.partial(get_ids_from_page, minPrice=minPrice, maxPrice=maxPrice, session=session), range(1, num_pages+ 1), max_workers=64)))

def get_property(id, session):
    property_url = f"http://www.immoweb.be/en/classified/{id}"
    try:
        tables = pd.read_html(session.get(property_url, timeout=5).text)
        df = pd.concat(tables).set_index(0).T
        df['id'] = id
        df = df.set_index('id')
        return df.loc[:, ~df.columns.duplicated()]
    except:
        return None

def get_properties(ids, session, max_workers=64):
    return pd.concat(thread_map(functools.partial(get_property, session=session), ids, max_workers=max_workers))

In [7]:
with requests.Session() as session:
    ids = get_price_range(0, 10**7, session)
    properties = get_properties(ids, session)

100%|██████████| 65313/65313 [17:22<00:00, 62.67it/s]  


In [8]:
properties.to_csv('properties.csv')